<a href="https://colab.research.google.com/github/Doaa-Said/cellula_intern/blob/main/Rag_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install chromadb sentence-transformers openai


In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer
from openai import OpenAI


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("openai_humaneval")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
chroma_client = chromadb.Client()


In [ ]:
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
        num_rows: 164
    })
})


In [ ]:
collection=chroma_client.create_collection(name="humanEval")


In [ ]:
prompt_embedding=embedding_model.encode( dataset["test"]["prompt"]).tolist()


In [ ]:
canonical_solution_embedding=embedding_model.encode(dataset["test"]["canonical_solution"]).tolist()


In [ ]:
collection.add(
    documents=list(dataset["test"]["prompt"]),                   # main searchable text = the prompt
    embeddings=prompt_embedding,        # vectors for prompts
    ids=[str(i) for i in dataset["test"]["task_id"]]      # unique IDs (as strings)
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# --------------------------
# 7. New programming task
# --------------------------
new_task = "Write a Python function to check if a string is a palindrome."

# Encode query & retrieve similar tasks
query_embedding = embedding_model.encode([new_task]).tolist()[0]
results = collection.query(query_embeddings=[query_embedding], n_results=3)

# --------------------------
# 8. Build context for RAG
# --------------------------
context = "\n\n".join(
    [f"Task: {d}" for d in results["documents"][0]]
)

prompt = f"""
You are a helpful coding assistant.

Here are some related tasks (short snippets only):
{context}

Now solve this new task:
{new_task}

Only provide the Python function code. Do NOT include any examples, docstrings, or extra text.
"""
# --------------------------
# 9. Generate free code with HF model
# --------------------------
output = generator(prompt, max_new_tokens=400, temperature=0.3, do_sample=True)

print("\n💡 Generated Code:\n")
print(output[0]["generated_text"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



💡 Generated Code:


You are a helpful coding assistant.

Here are some related tasks (short snippets only):
Task: 

def is_palindrome(text: str):
    """
    Checks if given string is a palindrome
    >>> is_palindrome('')
    True
    >>> is_palindrome('aba')
    True
    >>> is_palindrome('aaaaa')
    True
    >>> is_palindrome('zbcd')
    False
    """


Task: 

def is_palindrome(string: str) -> bool:
    """ Test if given string is a palindrome """
    return string == string[::-1]


def make_palindrome(string: str) -> str:
    """ Find the shortest palindrome that begins with a supplied string.
    Algorithm idea is simple:
    - Find the longest postfix of supplied string that is a palindrome.
    - Append to the end of the string reverse of a string prefix that comes before the palindromic suffix.
    >>> make_palindrome('')
    ''
    >>> make_palindrome('cat')
    'catac'
    >>> make_palindrome('cata')
    'catac'
    """


Task: 
def reverse_delete(s,c):
    """Task
    We 